# Read NWM 1km gridded output using the new parquet references approach

You need `fastparquet` and the `fsspec` from master (or a release after 3/30/2023)

I have this in my `environment.yml` file:
```
name: pangeo
channels:
- conda-forge
dependencies:
- python=3.10
- ...
- fastparquet
- pip:
  - fsspec @ git+https://github.com/fsspec/filesystem_spec@master
```

In [1]:
import fsspec
import xarray as xr

In [2]:
%%time
# references are on an OSN pod (no credentials needed)
url = 's3://rsignellbucket2/noaa/nwm/grid1km/refs/'

target_opts = {'anon':True, 'skip_instance_cache':True,
              'client_kwargs': {'endpoint_url': 'https://renc.osn.xsede.org'}}

# netcdf files are on the AWS public dataset program (no credentials needed)
remote_opts = {'anon':True}

fs = fsspec.filesystem("reference", fo=url, 
                       remote_protocol='s3', remote_options=remote_opts,
                      target_options=target_opts)
m = fs.get_mapper("")

CPU times: user 303 ms, sys: 55.7 ms, total: 358 ms
Wall time: 1.54 s


In [3]:
%%time
ds = xr.open_dataset(m, engine='zarr', chunks={'time':1, 'y':3840, 'x':4608})

CPU times: user 5.21 s, sys: 411 ms, total: 5.62 s
Wall time: 8.64 s


In [ ]:
ds

In [ ]:
ds['ACCET']

In [ ]:
import hvplot.xarray

In [ ]:
da = ds['ACCET'].sel(time='2019-01-01 12:00').load()

In [ ]:
da.hvplot.quadmesh(x='x', y='y', rasterize=True, data_aspect=1)